# Building Open Source Geochemical Research Tools in Python

<a href="https://doi.org/10.5281/zenodo.3874952"><img src="https://zenodo.org/badge/DOI/10.5281/zenodo.3874952.svg" align="right" alt="doi: 10.5281/zenodo.3875779" style="padding: 0px 10px 10px 0px"></a>
<a href="https://github.com/morganjwilliams/gs2020-python4geochem/blob/master/LICENSE"><img src="https://img.shields.io/badge/License-MIT-blue.svg" align="right" alt="License: MIT" style="padding: 0px 10px 10px 0px"></a>

<span id='authors'><b>Morgan Williams <a class="fa fa-twitter" aria-hidden="true" href="https://twitter.com/metasomite" title="@metasomite"></a></b>, Louise Schoneveld, Steve Barnes and Jens Klump; </span>
<span id='affiliation'><em>CSIRO Mineral Resources</em></span>

### Contents

| [**Abstract**](./00_overview.ipynb) | **Introduction**  | [**Examples**](./00_overview.ipynb#Examples)  |
|:-----|:-----|:-----|
|  | [Software in Geochem](./01_intro.ipynb#Software-in-Geochemistry)  |  [pyrolite](./011_pyrolite.ipynb)  |
|  | [Development & Tools](./01_intro.ipynb#Development-Workflow-&-Tools) | [pyrolite-meltsutil](./012_pyrolite-meltsutil.ipynb) | 
|  |  | [interferences](./013_interferences.ipynb) | 
|  |  | [autopew](./014_autopew.ipynb) |  

## interferences

>  Tools for inorganic mass spectra and interference patterns.

[![Docs](https://readthedocs.org/projects/interferences/badge/?version=develop)](https://interferences.readthedocs.io/)

This is an under-development package to facilitate debugging and identification of
issues during method development and analysis of novel samples for geologically-focused
mass spectrometry.

``interferences`` is centred around building tables which contain sets of molecular ions corresponding to a specific set of elements within your analytical target. Building the tables themselves is relatively straightforward. For examples, to build a table of ions which might be expected from Ca, O, Ar and H with a charge of +1 and up to two atoms within a molecule, you could use:

```python
df = build_table(["Ca", "O", "Ar", "H"], charges=[1], max_atoms=2)
```

These tables can then be displayed graphically, with a rudimentary estimate of relative abundance shown. Currently, ``interferences`` has two such methods, ``stemplot`` and ``spectra``. The first simply illustates the relative position of peaks (in m/z), while the second attempts to represent the width of some of these mass peaks given a specified mass resolution.

In [ ]:
import pandas as pd
import numpy as np
from interferences.table import build_table
import matplotlib.pyplot as plt
from pyrolite.geochem.ind import REE

In [ ]:
window = ("Tm[169]", 0.1)
df = build_table(REE() + ["O", "N", "H"], window=window, max_atoms=2)
ax = df.mz.stemplot(window=window, max_labels=5, figsize=(8, 4))
ax.figure.suptitle('stemplot: Tm Interferences')
plt.show()

In [ ]:
window = ("B[10]O[16]", 0.05)
df = build_table(["C", "B", "N", "O"], window=window, max_atoms=2)
ax = df.mz.spectra(window=window, mass_resolution=3000, max_labels=5, figsize=(8, 4))
ax.figure.suptitle('spectra: Cyanide BO Interference')
plt.show()

We've included some interactive examples below, if you'd like to play around with some of these without getting into the code itself. Note that it takes a second or two to run and refresh on these servers.

In [ ]:
import ipywidgets as widgets
#interact_manual = widgets.interact.options(manual=True, manual_name="Build Plot")

In [ ]:
def show_table(elements=[],
              window1=None,
              window2=0.05,
              max_atoms=2,
              max_charge=2):
    try:

        elements = [el.strip() for el in elements.strip(',').split(',')]

        if window1:
             window = (window1, window2)
        else:
            window=None
        df = build_table(elements,
                         window=window,
                         max_atoms=max_atoms,
                         charges=[i+1 for i in range(max_charge)])
        print("Table size: {}".format(df.index.size))
        display(df.style.background_gradient(cmap="Blues",
                                             axis=0,
                                             subset=pd.IndexSlice[:, ['iso_product']])
               )
    except:
        pass
mode= widgets.ToggleButtons(options=['spectra','stemplot'], value='spectra', description='Mode:')
elements=widgets.Text(value="C,B,N,O",description='Elements:')
max_atoms=widgets.IntSlider(min=1, max=3, step=1, value=2,
                            description='Atoms:', continuous_update=False)
max_charge=widgets.IntSlider(min=1, max=3, step=1, value=2,
                             description='Max Charge:', continuous_update=False )

window1=widgets.Text(value="B[10]O[16]", description='Target:')
window2=widgets.FloatLogSlider(min=-2, max=np.log10(2), step=0.1, value=0.05,
                               description='Mass Width', continuous_update=False)
mass_resolution=widgets.IntSlider(min=500, max=10000, step=1000, value=3000,
                                  description='Resolution:', continuous_update=False)
image_ratio = widgets.FloatSlider(min=0, max=1.5, step=0.1, value=0.5,
                                  description='Image Ratio:', continuous_update=False)


tableui = widgets.VBox([widgets.HTML(value="<h3>Table Generator</h3>"),
                        widgets.HBox([widgets.VBox([elements, max_atoms, max_charge]),
                                       widgets.VBox([window1, window2])
                                     ])])

tableout = widgets.interactive_output(show_table, {
                                     'elements': elements, 
                                     'max_atoms': max_atoms,
                                     'max_charge': max_charge, 
                                     'window1': window1, 
                                     'window2': window2
                                })
tableout.layout.width='500px'
display(tableui, tableout)

--------

In [ ]:
def plot_function(elements=[],
                  window1=None,
                  window2=0.05,
                  max_atoms=2,
                  max_charge=2,
                  mass_resolution=3000,
                  image_ratio=0,
                  n_labels=2):
    try:
        elements = [el.strip() for el in elements.strip(',').split(',')]
        window = (window1, window2)
        df = build_table(elements,
                         window=window,
                         max_atoms=max_atoms,
                         charges=[i+1 for i in range(max_charge)])
        ax = df.mz.spectra(window=window,
                               mass_resolution=mass_resolution,
                               image_ratio=image_ratio, 
                               max_labels=n_labels, 
                               figsize=(8, 4),
                              iter_lim=10)
    except:
        pass

elements=widgets.Text(value="C,B,N,O",description='Elements:')
max_atoms=widgets.IntSlider(min=1, max=3, step=1, value=2,
                            description='Atoms:', continuous_update=False)
max_charge=widgets.IntSlider(min=1, max=3, step=1, value=2,
                             description='Max Charge:', continuous_update=False )
n_labels= widgets.IntSlider(min=0, max=5, step=1, value=4,
                             description='# Labels:', continuous_update=False )


window1=widgets.Text(value="B[10]O[16]", description='Target:')
window2=widgets.FloatLogSlider(min=-2, max=0, step=0.1, value=0.05,
                               description='Mass Width', continuous_update=False)
mass_resolution=widgets.IntSlider(min=1000, max=15000, step=2000, value=3000,
                                  description='Resolution:', continuous_update=False)
image_ratio = widgets.FloatSlider(min=0, max=1.5, step=0.1, value=0.5,
                                  description='Image Ratio:', continuous_update=False)

ui = widgets.VBox([widgets.HTML(value="<h3>Spectra Generator</h3>"),
                   widgets.HBox([
                           widgets.VBox([elements, max_atoms, max_charge, n_labels]),
                           widgets.VBox([window1, window2, mass_resolution, image_ratio])
                   ])
                  ])

out = widgets.interactive_output(plot_function, {'n_labels': n_labels, 
                                     'elements': elements, 
                                     'max_atoms': max_atoms,
                                     'max_charge': max_charge, 
                                     'window1': window1, 
                                     'window2': window2,
                                     'mass_resolution': mass_resolution,
                                     'image_ratio' :image_ratio
                                })
out.layout.height = '350px'

display(ui, out)

If you're interested in seeing some more of this, check out the [documentation](https://interferences.readthedocs.io/) or [repository](https://github.com/morganjwilliams/interferences).

-----

| [**Abstract**](./00_overview.ipynb) | **Introduction**  | [**Examples**](./00_overview.ipynb#Examples)  |
|:-----|:-----|:-----|
|  | [Software in Geochem](./01_intro.ipynb#Software-in-Geochemistry)  |  [pyrolite](./011_pyrolite.ipynb)  |
|  | [Development & Tools](./01_intro.ipynb#Development-Workflow-&-Tools) | [pyrolite-meltsutil](./012_pyrolite-meltsutil.ipynb) | 
|  |  | [interferences](./013_interferences.ipynb) | 
|  |  | [autopew](./014_autopew.ipynb) |  